In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-30"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8103,2024-05-30,Eua Nba,21:30,Minnesota Timberwolves,Dallas Mavericks,1.51,2.67,208.5,1.81,2.04,-4.5,1.89,3.70,2V13yfaE,0.662252,0.374532,0.552486,0.490196,0.036783,0.6,0.4,NaN,NaN,186.340,39.478140,0.211861,218.082,45.332629,0.207870,172.622,227.192,164.85,272.16,0.0,0.0,0.0,0.0,0.392461,0.084485,0.457912,-1.56,-0.312,-1.634615,0.569134,0.4,-0.169134,3.50,0.700,2.385714,0.519464,0.9,0.380536
8104,2024-05-30,Itália Liga A,15:45,Brescia,Milano,2.24,1.66,157.5,1.84,1.92,2.5,1.92,1.42,0fDe1T7c,0.446429,0.602410,0.543478,0.520833,0.048838,0.2,0.4,NaN,NaN,166.902,77.223406,0.462687,117.164,36.669909,0.312979,128.458,121.506,147.46,114.48,0.0,0.0,0.0,0.0,0.210319,0.030090,0.211709,0.76,0.152,8.157895,0.000000,0.0,0.000000,-1.01,-0.202,-3.267327,0.710212,0.5,-0.210212
8105,2024-05-30,Bulgária Nbl,13:00,Balkan,Rilski Sportist,2.02,1.72,157.5,1.87,1.79,1.5,1.88,1.59,riOymvN6,0.495050,0.581395,0.534759,0.558659,0.076445,0.8,1.0,NaN,NaN,139.172,33.561685,0.241153,129.132,21.477287,0.166320,136.770,132.496,159.80,155.70,0.0,0.0,0.0,0.0,0.113440,0.030912,0.118191,1.05,0.210,4.857143,0.000000,0.0,0.000000,-0.74,-0.148,-4.864865,0.669764,0.7,0.030236
8106,2024-05-30,Eua Wnba,20:00,Indiana Fever F,Seattle Storm F,2.17,1.70,166.5,1.81,1.95,2.5,1.96,1.43,QDCWdrTR,0.460829,0.588235,0.552486,0.512821,0.049065,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,204.12,158.10,0.0,0.0,0.0,0.0,0.171752,0.052657,0.221101,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8107,2024-05-30,Eua Wnba,21:00,Chicago Sky F,Los Angeles Sparks F,1.62,2.31,163.5,1.83,1.93,-4.5,1.99,2.85,SMbbh2yq,0.617284,0.432900,0.546448,0.518135,0.050184,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.248297,0.037612,0.251286,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8108,2024-05-30,França Lnb,16:05,Lyon-Villeurbanne,Paris,1.66,2.24,162.5,1.82,1.96,-3.5,1.94,3.00,KfAbAN1M,0.602410,0.446429,0.549451,0.510204,0.048838,0.4,0.4,NaN,NaN,148.236,57.550746,0.388237,125.044,31.863083,0.254815,106.902,146.434,120.12,140.00,0.0,0.0,0.0,0.0,0.210319,0.052378,0.303455,1.07,0.214,3.084112,0.000000,0.0,0.000000,-3.25,-0.650,-1.907692,0.000000,0.0,0.000000
8109,2024-05-30,Países Baixos Dbl,15:00,Leiden,Den Bosch,1.83,1.89,152.5,1.86,1.80,-1.5,1.96,2.05,fXQ7T1in,0.546448,0.529101,0.537634,0.555556,0.075549,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.022810,0.023184,0.031740,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8110,2024-05-30,Porto Rico Bsn,21:00,Gigantes de Carolina,Cangrejeros,2.12,1.67,182.5,1.82,1.88,-1.5,1.87,1.74,8lQ6UIXN,0.471698,0.598802,0.549451,0.531915,0.070501,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,147.92,193.14,0.0,0.0,0.0,0.0,0.167915,0.022933,0.050927,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8111,2024-05-30,Rússia Liga Vtb United,13:30,Lokomotiv Kuban,Zenit Petersburg,1.69,2.19,165.5,1.80,1.99,-2.5,2.02,2.57,CzOkAQwo,0.591716,0.456621,0.555556,0.502513,0.048337,1.0,0.6,NaN,NaN,179.888,39.719256,0.220800,156.854,27.719699,0.176723,144.238,145.554,151.30,195.65,0.0,0.0,0.0,0.0,0.182244,0.070897,0.169459,-0.31,-0.062,-11.129032,0.699065,0.8,0.100935,-0.86,-0.172,-6.918605,0.635917,0.6,-0.035917


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:30,Eua Nba,Minnesota Timberwolves,Dallas Mavericks,208.5,1.81,1.0,Over
1,21:00,Eua Wnba,Chicago Sky F,Los Angeles Sparks F,163.5,1.83,1.0,Over
2,16:05,França Lnb,Lyon-Villeurbanne,Paris,162.5,1.82,1.0,Over
3,15:00,Países Baixos Dbl,Leiden,Den Bosch,152.5,1.86,1.0,Over
4,13:30,Rússia Liga Vtb United,Lokomotiv Kuban,Zenit Petersburg,165.5,1.80,1.0,Over
